# Data Exploration


Read the propagandist users' data and apply fixes to the columns' names and values.


## Read Data


In [1]:
import re
import emoji
import pandas as pd
from tqdm.auto import tqdm
from arabert.preprocess import ArabertPreprocessor

In [2]:
raw_data_path = "../data/raw/propaganda.csv"
processed_data_path = "../data/processed/propaganda.json"
processing_model = "aubmindlab/bert-base-arabertv2"

In [3]:
arabert_prep = ArabertPreprocessor(model_name=processing_model)

[2023-11-12 02:09:54,729 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [4]:
df = pd.read_csv(raw_data_path, parse_dates=["account_creation_date"])
df.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,tweet_text,is_retweet,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions
0,1161436140945195008,392352672,نجديه عذيه .,Ksa_FO1,نجد العذيه للدعم,مابادل اقل من ٥ الاف ولا أقبل المقدم بدون طلب ...,NaN,70616,52577,2011-10-16,RT @GROB_07: #حـسـآب_تـمــيزة_بـتـفـاعـل 💫\n⠀┈...,True,0.0,0.0,0.0,0.0,"['حـسـآب_تـمــيزة_بـتـفـاعـل', 'يوسف', 'الاصدق...",[],"['975702084787015680', '864364254790537216']"
1,764273648496742400,4708503082,ْ‏ོ جود العبداللّهہ|| ོالتنبيهات ما توصل,j06__,NaN,‏‏‏‏‏‏‏‏‏‏‏إنه يستجيب منذُ أن دعوُت ولگِن يُدب...,NaN,60061,2617,2016-01-04,اللهم اكثر من امثالي ليعم الخير و تعم السعادة ...,False,0.0,0.0,0.0,0.0,['مسابقات_اعلان'],[],[]


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   tweetid                   56000 non-null  int64         
 1   userid                    56000 non-null  object        
 2   user_display_name         56000 non-null  object        
 3   user_screen_name          56000 non-null  object        
 4   user_reported_location    36366 non-null  object        
 5   user_profile_description  51749 non-null  object        
 6   user_profile_url          9912 non-null   object        
 7   follower_count            56000 non-null  int64         
 8   following_count           56000 non-null  int64         
 9   account_creation_date     56000 non-null  datetime64[ns]
 10  tweet_text                56000 non-null  object        
 11  is_retweet                56000 non-null  bool          
 12  quote_count       

## Fix Int Data


In [6]:
float_cols = ["quote_count", "reply_count", "like_count", "retweet_count"]

for col in tqdm(float_cols):
    df[col] = df[col].astype(int)

  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   tweetid                   56000 non-null  int64         
 1   userid                    56000 non-null  object        
 2   user_display_name         56000 non-null  object        
 3   user_screen_name          56000 non-null  object        
 4   user_reported_location    36366 non-null  object        
 5   user_profile_description  51749 non-null  object        
 6   user_profile_url          9912 non-null   object        
 7   follower_count            56000 non-null  int64         
 8   following_count           56000 non-null  int64         
 9   account_creation_date     56000 non-null  datetime64[ns]
 10  tweet_text                56000 non-null  object        
 11  is_retweet                56000 non-null  bool          
 12  quote_count       

## Fix List Data


In [8]:
list_cols = ["hashtags", "urls", "user_mentions"]

for col in tqdm(list_cols):
    df[col] = [eval(v) if not pd.isna(v) else [] for v in df[col].values]

  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   tweetid                   56000 non-null  int64         
 1   userid                    56000 non-null  object        
 2   user_display_name         56000 non-null  object        
 3   user_screen_name          56000 non-null  object        
 4   user_reported_location    36366 non-null  object        
 5   user_profile_description  51749 non-null  object        
 6   user_profile_url          9912 non-null   object        
 7   follower_count            56000 non-null  int64         
 8   following_count           56000 non-null  int64         
 9   account_creation_date     56000 non-null  datetime64[ns]
 10  tweet_text                56000 non-null  object        
 11  is_retweet                56000 non-null  bool          
 12  quote_count       

## Clean Text


In this stage we do multiple steps as follows:

- Remove all emojis
- Remove all mention and hashtag symbols
- Remove all links and mentions

The cleaned text will be saved in the field `text` in the data frame and will be the main feature from now on.


In [10]:
# def process_text(text):
#     """Process tweet text by removing links, mentions, and hashtags symbol."""
#     # links
#     clean_text = re.sub(r"http\S+|t\.co/\S+", "", text)
#     # mentions
#     clean_text = re.sub(r"@\w+", "", clean_text)
#     # hashtags
#     clean_text = re.sub(r"#", "", clean_text)
#     clean_text = re.sub(r"_", " ", clean_text)
#     # tashqeel - from @bakriano
#     clean_text = re.sub(r"[\u0617-\u061A\u064B-\u0652]", "", clean_text)
#     # emojis
#     clean_text = emoji.replace_emoji(clean_text, replace="")
#     # remove new lines and normalize white spaces
#     clean_text = re.sub(r"\s+", " ", clean_text)
#     return clean_text.replace("RT :", "").strip()

In [11]:
def process_text(text):
    """Process tweet text by removing links, mentions, and hashtags symbol."""
    clean_text = arabert_prep.preprocess(text)
    clean_text = arabert_prep.unpreprocess(clean_text)
    clean_text = clean_text.replace("[رابط]", "")
    clean_text = clean_text.replace("[مستخدم]", "")
    clean_text = clean_text.replace("RT", "")
    clean_text = clean_text.replace(":", "")
    return clean_text.strip()

In [12]:
df["text"] = df["tweet_text"].apply(process_text)

In [13]:
df.head(2)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,tweet_text,is_retweet,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,text
0,1161436140945195008,392352672,نجديه عذيه .,Ksa_FO1,نجد العذيه للدعم,مابادل اقل من ٥ الاف ولا أقبل المقدم بدون طلب ...,NaN,70616,52577,2011-10-16,RT @GROB_07: #حـسـآب_تـمــيزة_بـتـفـاعـل 💫\n⠀┈...,True,0,0,0,0,"[حـسـآب_تـمــيزة_بـتـفـاعـل, يوسف, الاصدقاء_لل...",[],"[975702084787015680, 864364254790537216]",# حسآب_تميزة_بتفاعل نج م القوآئم لهذآ اليوم # ...
1,764273648496742400,4708503082,ْ‏ོ جود العبداللّهہ|| ོالتنبيهات ما توصل,j06__,NaN,‏‏‏‏‏‏‏‏‏‏‏إنه يستجيب منذُ أن دعوُت ولگِن يُدب...,NaN,60061,2617,2016-01-04,اللهم اكثر من امثالي ليعم الخير و تعم السعادة ...,False,0,0,0,0,[مسابقات_اعلان],[],[],اللهم أكثر من امثالي ليعم الخير و تعم السعادة#...


## Process List Columns


In [14]:
type(df.loc[0, "urls"])

list

In [15]:
df["urls"] = [len(urls) for urls in df["urls"].values]
df["hashtags"] = [len(hashtags) for hashtags in df["hashtags"].values]
df["user_mentions"] = [len(mentions) for mentions in df["user_mentions"].values]

## Drop Unnecessary Columns


In [16]:
df = df.drop(columns=["user_profile_url"])

## Save Clean Data


In [17]:
df.to_json(processed_data_path)